In [30]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType
from pyspark.sql.functions import regexp_replace,row_number
from pyspark.sql.window import Window

In [3]:
spark = (SparkSession
 .builder
 .appName("Profession")
 .getOrCreate())

22/08/27 03:48:17 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.197.132 instead (on interface ens33)
22/08/27 03:48:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/27 03:48:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/27 03:48:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Providing the Schema for the DataFrame.
Two Columns Salary and Profession

In [16]:
schema = StructType([
             StructField('Profession', StringType(), True),
             StructField('Salary', StringType(), True)
         ])

In [17]:
df =  spark.read.option('header','True').schema(schema).csv('German_Salaries_By_Profession.csv')

In [18]:
df.printSchema()

root
 |-- Profession: string (nullable = true)
 |-- Salary: string (nullable = true)



In [19]:
df.show()

+--------------------+-------+
|          Profession| Salary|
+--------------------+-------+
|              Doctor|€89,539|
|             Finance|€73,847|
|              Lawyer|€68,642|
|     Business Advice|€64,173|
|             Banking|€62,744|
|            Engineer|€62,564|
|                  IT|€60,563|
|       Marketing, PR|€60,174|
|          Production|€60,046|
|Distribution and ...|€59,591|
|                  HR|€58,029|
|        Construction|€57,270|
|            Research|€56,399|
|           Logistics|€50,358|
|              Design|€48,574|
+--------------------+-------+



The Salary Column had some Euro symbols and Comma needed to remove that so that it can be inferred correctly moving forward

In [23]:
df = df.withColumn('Salary',regexp_replace(df.Salary,pattern='[^a-zA-Z0-9]', replacement=''))

Did not have any id column. So, used Row_number function to generate some ids for that

In [31]:
window = Window.orderBy('Salary')
df = df.withColumn('Id',row_number().over(window))

df.show(truncate=False)

22/08/27 04:36:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 04:36:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------------+------+---+
|Profession            |Salary|Id |
+----------------------+------+---+
|Design                |48574 |1  |
|Logistics             |50358 |2  |
|Research              |56399 |3  |
|Construction          |57270 |4  |
|HR                    |58029 |5  |
|Distribution and Sales|59591 |6  |
|Production            |60046 |7  |
|Marketing, PR         |60174 |8  |
|IT                    |60563 |9  |
|Engineer              |62564 |10 |
|Banking               |62744 |11 |
|Business Advice       |64173 |12 |
|Lawyer                |68642 |13 |
|Finance               |73847 |14 |
|Doctor                |89539 |15 |
+----------------------+------+---+



In [60]:
df.write.format('csv').option('header','true').mode('overwrite').save('Profession')

22/08/27 04:58:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 04:58:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 04:58:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [37]:
df.rdd.getNumPartitions()

1